In [32]:
import pandas as pd
import numpy as np
import joblib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.transformation import LogTransformer 
from feature_engine.selection import DropFeatures

# Cargar operadores definidos
import operators

# 1. Cargamos dataset

In [33]:
# Cargar el dataset
data_train= pd.read_csv('../data/raw/stores_sales_forecasting_updated_v3.1.csv', 
                 sep=';',
                 encoding='utf-8')

# Convertir fechas
data_train['Order Date'] = pd.to_datetime(data_train['Order Date'], dayfirst=True, errors='coerce')
data_train['Ship Date'] = pd.to_datetime(data_train['Ship Date'], dayfirst=True, errors='coerce')

# Variables derivadas
data_train['Order_Month'] = data_train['Order Date'].dt.month
data_train['Order_Quarter'] = data_train['Order Date'].dt.quarter
data_train['Days to Ship'] = (data_train['Ship Date'] - data_train['Order Date']).dt.days

# Asegurar que sea numérico para evitar errores
data_train['Postal Code'] = pd.to_numeric(data_train['Postal Code'], errors='coerce')
data_train['Discount'] = pd.to_numeric(data_train['Discount'], errors='coerce')
data_train['Quantity'] = pd.to_numeric(data_train['Quantity'], errors='coerce')
data_train['Profit'] = pd.to_numeric(data_train['Profit'], errors='coerce')
data_train['Order_Month'] = pd.to_numeric(data_train['Order_Month'], errors='coerce')
data_train['Order_Quarter'] = pd.to_numeric(data_train['Order_Quarter'], errors='coerce')
data_train['Days to Ship'] = pd.to_numeric(data_train['Days to Ship'], errors='coerce')

data_train.head(5)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Order_Month,Order_Quarter,Days to Ship
0,849,CA-2017-107503,2017-01-01,2017-01-06,Standard Class,GA-14725,Guy Armstrong,Consumer,United States,Lorain,...,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",48.896,4,0.2,8.5568,1,1,5
1,4010,CA-2017-144463,2017-01-01,2017-01-05,Standard Class,SC-20725,Steven Cartwright,Consumer,United States,Los Angeles,...,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Brentwood Wall ...",474.430,11,0.0,199.2606,1,1,4
2,8071,CA-2017-151750,2017-01-01,2017-01-05,Standard Class,JM-15250,Janet Martin,Consumer,United States,Huntsville,...,Furniture,Furnishings,"Tenex Carpeted, Granite-Look or Clear Contempo...",141.420,5,0.6,-187.3815,1,1,4
3,8072,CA-2017-151750,2017-01-01,2017-01-05,Standard Class,JM-15250,Janet Martin,Consumer,United States,Huntsville,...,Furniture,Chairs,Office Star - Contemporary Task Swivel Chair,310.744,4,0.3,-26.6352,1,1,4
4,867,CA-2014-149020,2014-01-10,2014-01-15,Standard Class,AJ-10780,Anthony Jacobs,Corporate,United States,Springfield,...,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Ridgewood Wall ...",51.940,1,0.0,21.2954,1,1,5


# 2. Train, Test Split

In [34]:
data_train = pd.read_csv('../data/raw/stores_sales_forecasting_updated_v3.1.csv', 
                         sep=';',
                         encoding='utf-8')

X = data_train.drop(['Sales'], axis=1)
y = data_train['Sales']

# ✅ Split temporal (80/20) - SIN shuffle
split_index = int(len(data_train) * 0.8)

x_train = X.iloc[:split_index].copy()
x_test = X.iloc[split_index:].copy()
y_train = y.iloc[:split_index].copy()
y_test = y.iloc[split_index:].copy()

In [35]:
# Si no hay valores nulos, crear algunos artificialmente para practicar
# Ejecutar una sola vez para no generar varios datos sinteticos
np.random.seed(42)

# Crear nulos en variables categóricas (Ship Mode, Segment, Sub-Category)
sample_size_cat = int(len(data_train) * 0.05)  # 3% de los datos
null_indices_ship = np.random.choice(data_train.index, sample_size_cat, replace=False)
data_train.loc[null_indices_ship, 'Ship Mode'] = np.nan

null_indices_segment = np.random.choice(data_train.index, sample_size_cat, replace=False)
data_train.loc[null_indices_segment, 'Segment'] = np.nan

null_indices_segment = np.random.choice(data_train.index, sample_size_cat, replace=False)
data_train.loc[null_indices_segment, 'Sub-Category'] = np.nan

# Crear nulos en variables numéricas ( Quantity, Discount)
sample_size_num = int(len(data_train) * 0.05)  # 5% de los datos
null_indices_quantity = np.random.choice(data_train.index, sample_size_num, replace=False)
data_train.loc[null_indices_quantity, 'Quantity'] = np.nan

null_indices_quantity = np.random.choice(data_train.index, sample_size_num, replace=False)
data_train.loc[null_indices_quantity, 'Discount'] = np.nan

print("Valores nulos creados artificialmente:")
print(data_train.isnull().sum())

Valores nulos creados artificialmente:
Row ID             0
Order ID           0
Order Date         0
Ship Date          0
Ship Mode        106
Customer ID        0
Customer Name      0
Segment          106
Country            0
City               0
State              0
Branch             0
Postal Code        0
Region             0
Product ID         0
Category           0
Sub-Category     106
Product Name       0
Sales              0
Quantity         106
Discount         106
Profit             0
dtype: int64


In [36]:
# Validacion de valores nulos
print("=== VARIABLES DISPONIBLES ===")
columnas_info = pd.DataFrame({
    'Variable': data_train.columns,
    'Tipo': data_train.dtypes.values,
    'Nulos': data_train.isnull().sum().values
})
print(columnas_info.to_string(index=False))

=== VARIABLES DISPONIBLES ===
     Variable    Tipo  Nulos
       Row ID   int64      0
     Order ID  object      0
   Order Date  object      0
    Ship Date  object      0
    Ship Mode  object    106
  Customer ID  object      0
Customer Name  object      0
      Segment  object    106
      Country  object      0
         City  object      0
        State  object      0
       Branch  object      0
  Postal Code   int64      0
       Region  object      0
   Product ID  object      0
     Category  object      0
 Sub-Category  object    106
 Product Name  object      0
        Sales float64      0
     Quantity float64    106
     Discount float64    106
       Profit float64      0


# 3. Configuración del Pipeline

In [37]:
#imputacion de variables categoricas con imputación por frecuencia
CATEGORICAL_VARS_WITH_NA_FREQUENT=['Sub-Category']

#Imputacion de variables númericas con imputacion por media
NUMERICAL_VARS_WITH_NA = ['Quantity', 'Discount']

# Imputación de variables categóricas con valor faltante (Missing)
CATEGORICAL_VARS_WITH_NA_MISSING = ['Segment']

# Variables a eliminar
DROP_FEATURES = ['Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Order Date', 'Ship Date', 'Branch', 'Postal Code',
                 'Product ID','Product Name']

# Variables para transformación logarítmica
NUMERICAL_LOG_VARS = ['Quantity']

# Variables para codificación ordinal (calidad)
QUAL_VARS = ['Ship Mode']

# Variables para codificación por frecuencia (no ordinal)
CATEGORICAL_VARS = ['Segment', 'Sub-Category', 'Country', 'City','State' , 'Region', 'Category']

# Mapeos para variables categóricas de calidad
QUAL_MAPPINGS = {'Standard Class': 1, 'Second Class': 2, 'First Class': 3}

# Variables numéricas principales
NUMERICAL_VARS = ['Quantity', 'Discount', 'Profit', 'Order_Month','Order_Quarter','Days to Ship']


# Variables finales para el modelo
FEATURES = ['Quantity', 'Discount', 'Profit', 'Ship Mode', 'Segment', 'Country', 'City',
            'State' , 'Region', 'Category','Sub-Category', 'Order_Month','Order_Quarter','Days to Ship']


# 4. Seleccion de Features de interes 

In [38]:
all_features=set(x_train.columns)
features_to_drop =all_features.difference(FEATURES)
features_to_drop = list(features_to_drop)

# 5. Construccion de Pipeline

In [39]:
stores_sales_forecasting_data_pre_proc= Pipeline([
#0. Seleccion de features para el modelo 
    ('drop_features',
            DropFeatures(features_to_drop=features_to_drop)),
#1. Imputacion de variables categoricas
    ('cat_missing_imputation',
            CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)),

#2. Imputacion de variables categoricas por frecuencia
    ('cat_missing_freq_imputation',
            CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)),

#3. Imputacion de variables númericas
    ('mean_imputation',
            MeanMedianImputer(imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)),

#4.Codificacion de variables categoricas
    ('quality_mapper',
            operators.Mapper(variables=QUAL_VARS, mappins=QUAL_MAPPINGS)),

#5. Codificacion por Frecuency encoding
    ('cat_freq_encode',
            CountFrequencyEncoder(encoding_method='count', variables=CATEGORICAL_VARS)),

#6.Transformacion de variables continuas
    ('continues_log_transform',
            LogTransformer(variables=NUMERICAL_LOG_VARS)),

#7. Normalizacion de variables
    ('Variable_scaler',
            MinMaxScaler())
])

# 6. Aplicamos Pipeline para Construccion de features

In [40]:
stores_sales_forecasting_data_pre_proc.fit(x_train, y_train)

,steps,"[('drop_features', ...), ('cat_missing_imputation', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,features_to_drop,"['Branch', 'Postal Code', ...]"
,imputation_method,'missing'
,fill_value,'Missing'
,variables,['Segment']
,return_object,False
,ignore_format,False
,imputation_method,'frequent'


In [41]:
def save_procesed_data(x,y, str_df_name):
    X_transformed = stores_sales_forecasting_data_pre_proc.transform(x)
    df_X_train_transformed= pd.DataFrame(data=X_transformed, columns=FEATURES)
    y = y.reset_index()
    df_transformed= pd.concat([df_X_train_transformed, y['Sales']], axis=1)
    df_transformed.to_csv(f"../data/interim/proc_{str_df_name}.csv", index=False)

# 7.1 Procesamos data de entrenamiento

In [44]:
save_procesed_data(x_train, y_train, str_df_name="data_train")

ValueError: Shape of passed values is (1696, 11), indices imply (1696, 14)

# 7.2 Procesamos data para test

In [13]:
save_procesed_data(x_test, y_test, str_df_name="data_test")

c:\Users\melvi\anaconda3\envs\venv_papd_a\Lib\site-packages\feature_engine\encoding\base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) City, State.
  warnings.warn(


ValueError: Shape of passed values is (425, 11), indices imply (425, 14)

# 8. Exportamos Pipeline de pre-procesamiento ya configurado

In [26]:
joblib.dump(stores_sales_forecasting_data_pre_proc, '../models/stores_sales_forecasting_data_pre_proc.pkl')

['../models/stores_sales_forecasting_data_pre_proc.pkl']